In [1]:
import tensorflow as tf

In [5]:
mnist = tf.keras.datasets.mnist

# 加载数据
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

# 增加维度， (60000, 28, 28) --> (60000, 28, 28, 1)
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

使用 tf.data 创建batch和shuffle的数据集

### tf.data 用于处理输入数据

#### Dataset 

支持编写高性能的数据输入管道，可以按照下面的方式来使用:
- 从自定义输入创建数据集
- 应用数据转换进行数据预处理
- 迭代式的处理数据（迭代式处理数据是目前流行的方式，避免全量加载到内存导致内存空间不足）

直接基于数据创建数据集:
```
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
for element in dataset:
    print(element)
```
基于文件创建数据集
```
dataset = tf.data.TextLineDataset(["file1.txt", "file2.txt"])
```
基于TFRecord格式创建数据集
```
dataset = tf.data.TFRecordDataset(["file1.tfrecords", "file2.tfrecords"])
```
基于通配符文件创建数据集
```
dataset = tf.data.Dataset.list_files("/path/*.txt")  # doctest: +SKIP
```

创建数据集后可以使用map进行转换操作:
```
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
dataset = dataset.map(lambda x: x*2)
list(dataset.as_numpy_iterator())
```

In [9]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

创建自定义模型

In [13]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, 3, activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.d1 = tf.keras.layers.Dense(128, activation='relu')
        self.d2 = tf.keras.layers.Dense(10)
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
model = MyModel()

In [19]:
# 选择损失函数和优化器
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [15]:
# 定义评价指标
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

训练模型

In [23]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_func(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [21]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_func(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [27]:
E = 5
for epoch in range(E):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
        
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
        
    template = 'Epoch {}, Loss {}, Accuracy {}, Test Loss {}, Test Accuracy {}'
    
    print(template.format((epoch+1), 
                          train_loss.result(), 
                          train_accuracy.result()*100, 
                          test_loss.result(), 
                          test_accuracy.result()*100))

Epoch 1, Loss 0.021291617304086685, Accuracy 99.32666778564453, Test Loss 0.05509920045733452, Test Accuracy 98.3699951171875
Epoch 2, Loss 0.01322136726230383, Accuracy 99.53333282470703, Test Loss 0.058831676840782166, Test Accuracy 98.29999542236328
Epoch 3, Loss 0.009120644070208073, Accuracy 99.6883316040039, Test Loss 0.05732549726963043, Test Accuracy 98.43000030517578
Epoch 4, Loss 0.00613583717495203, Accuracy 99.79000091552734, Test Loss 0.07237181812524796, Test Accuracy 98.11000061035156
Epoch 5, Loss 0.006463431753218174, Accuracy 99.78666687011719, Test Loss 0.08328071236610413, Test Accuracy 98.19999694824219


In [28]:
model.summary()

Model: "my_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            multiple                  320       
_________________________________________________________________
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  2769024   
_________________________________________________________________
dense_3 (Dense)              multiple                  1290      
Total params: 2,770,634
Trainable params: 2,770,634
Non-trainable params: 0
_________________________________________________________________
